In [ ]:
import requests
import csv
import time

# Use your API key
API_KEY = " LJS49EB0EZZGLCB9"

# Alpha Vantage limits results per request. Fetching in multiple batches.
BATCH_SIZE = 1000  # Fetch 1000 articles per request
TOTAL_ARTICLES = 2000  # Target number of articles
all_articles = []
request_count = 0

# Loop to fetch articles in multiple requests
while len(all_articles) < TOTAL_ARTICLES:
    url = f"https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers=AAPL&apikey={API_KEY}&sort=LATEST&limit={BATCH_SIZE}"
    
    # Make the request
    r = requests.get(url)
    data = r.json()
    
    # Check if news data exists
    if "feed" in data:
        articles = data["feed"]
        all_articles.extend(articles)  # Add new articles to the list
    else:
        print("Error: No articles found or API limit reached.")
        break  # Stop if no data is returned

    request_count += 1
    print(f"Fetched {len(all_articles)} articles so far...")

    # Avoid hitting API rate limits (Alpha Vantage limits requests per minute)
    if request_count % 5 == 0:
        print("Pausing to avoid rate limits...")
        time.sleep(60)  # Sleep for 60 seconds after every 5 requests

# Trim to exactly 2000 articles
all_articles = all_articles[:TOTAL_ARTICLES]

# Save to CSV
csv_filename = "news_articles_2000.csv"
with open(csv_filename, mode="w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Title", "Summary", "Published Date", "URL", "Sentiment Score", "Sentiment Label"])

    for article in all_articles:
        writer.writerow([
            article.get("title", "N/A"),
            article.get("summary", "N/A"),
            article.get("time_published", "N/A"),
            article.get("url", "N/A"),
            article.get("overall_sentiment_score", "N/A"),
            article.get("overall_sentiment_label", "N/A"),
        ])

print(f"✅ Successfully saved {len(all_articles)} news articles to {csv_filename}")
